In [1]:
# Imports
import numpy as np
import pandas as pd
import muon as mu
import mofax
import gget
import gseapy as gp

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Define the data and figure folder.
data_folder = "/users/csb/huizing/Documents/PhD/Code/mowgli_reproducibility/data/"
h_folder = "/users/csb/huizing/Documents/PhD/Code/Mowgli/local_analysis/from_jz/h/"

In [3]:
# Load the data.
mdata = mu.read_h5mu(data_folder + "TEA/tea_preprocessed.h5mu.gz")

In [4]:
# Load MOFA+'s weights.
mofa_model = mofax.mofa_model(data_folder + "TEA/tea_mofa_15.hdf5")
H_mofa = mofa_model.get_weights("rna")

In [5]:
# Load Mowgli's weights.
H_mowgli = np.load(
    h_folder + "tea_mowgli_cosine_50_0_05_rna_0_01_atac_0_1_adt_0_01_0_001.npy",
    allow_pickle=True,
).item()["H_rna"]

In [6]:
def rnk_mowgli(dim):
    """
    Get the genes ranks for a given dimension.
    """
    idx = H_mowgli[:, dim].argsort()[::-1]
    var_names = mdata["rna"].var_names[idx].str.replace("rna:", "").to_list()
    values = H_mowgli[idx, dim]
    return pd.Series(values, index=var_names)


def rnk_mofa(dim):
    """
    Get the genes ranks for a given dimension.
    """
    idx = H_mowgli[:, dim].argsort()[::-1]
    var_names = mdata["rna"].var_names[idx].str.replace("rna:", "").to_list()
    values = H_mowgli[idx, dim]
    return pd.Series(values, index=var_names)

In [15]:
gmt_folder = "/users/csb/huizing/Documents/PhD/Code/mowgli_reproducibility/enrich/gmts/"

sources = [
    gmt_folder + "GO_Biological_Process_2021.gmt",
    gmt_folder + "GO_Cellular_Component_2021.gmt",
    gmt_folder + "GO_Molecular_Function_2021.gmt",
    gmt_folder + "KEGG_2021_Human.gmt",
    gmt_folder + "PanglaoDB_Augmented_2021.gmt",
    gmt_folder + "Reactome_2016.gmt",
]


In [19]:
enr_total = pd.DataFrame({})

for source in sources:
    print(source)
    for dim in range(H_mowgli.shape[1]):
        print("mowgli", dim)
        enr = gp.prerank(
            rnk=rnk_mowgli(dim),
            gene_sets=source,
            min_size=15,
            processes=4,
            max_size=500,
            permutation_num=250,
            outdir=None,
            seed=42,
            verbose=False,
        ).res2d
        enr["dim"] = dim
        enr["source"] = source
        enr["method"] = "mowgli"
        enr["query"] = f"mowgli {dim}"
        enr_total = pd.concat([enr_total, enr])

    for dim in range(H_mofa.shape[1]):
        print("mofa", dim)
        enr = gp.prerank(
            rnk=rnk_mofa(dim),
            gene_sets=source,
            min_size=15,
            processes=4,
            max_size=500,
            permutation_num=250,
            outdir=None,
            seed=42,
            verbose=False,
        ).res2d
        enr["dim"] = dim
        enr["source"] = source
        enr["method"] = "mofa"
        enr["query"] = f"mofa {dim}"
        enr_total = pd.concat([enr_total, enr])


Wed Sep  7 12:16:50 2022 INFO Parsing data files for GSEA.............................


/users/csb/huizing/Documents/PhD/Code/mowgli_reproducibility/enrich/gmts/GO_Biological_Process_2021.gmt
mowgli 0


Wed Sep  7 12:17:03 2022 INFO 5730 gene_sets have been filtered out when max_size=500 and min_size=15
Wed Sep  7 12:17:03 2022 INFO 0306 gene_sets used for further statistical testing.....
Wed Sep  7 12:17:03 2022 INFO Start to run GSEA...Might take a while..................
Wed Sep  7 12:17:17 2022 INFO Start to generate gseapy reports, and produce figures...
Wed Sep  7 12:17:17 2022 INFO Congratulations. GSEApy runs successfully................

Wed Sep  7 12:17:17 2022 INFO Parsing data files for GSEA.............................


mowgli 1


Wed Sep  7 12:17:32 2022 INFO 5730 gene_sets have been filtered out when max_size=500 and min_size=15
Wed Sep  7 12:17:32 2022 INFO 0306 gene_sets used for further statistical testing.....
Wed Sep  7 12:17:32 2022 INFO Start to run GSEA...Might take a while..................
Wed Sep  7 12:17:46 2022 INFO Start to generate gseapy reports, and produce figures...
Wed Sep  7 12:17:46 2022 INFO Congratulations. GSEApy runs successfully................

Wed Sep  7 12:17:46 2022 INFO Parsing data files for GSEA.............................


...

Wed Sep  7 12:18:00 2022 INFO 5730 gene_sets have been filtered out when max_size=500 and min_size=15
Wed Sep  7 12:18:00 2022 INFO 0306 gene_sets used for further statistical testing.....
Wed Sep  7 12:18:00 2022 INFO Start to run GSEA...Might take a while..................
Wed Sep  7 12:18:14 2022 INFO Start to generate gseapy reports, and produce figures...
Wed Sep  7 12:18:14 2022 INFO Congratulations. GSEApy runs successfully................

Wed Sep  7 12:18:14 2022 INFO Parsing data files for GSEA.............................


mowgli 3


Wed Sep  7 12:18:27 2022 INFO 5730 gene_sets have been filtered out when max_size=500 and min_size=15
Wed Sep  7 12:18:27 2022 INFO 0306 gene_sets used for further statistical testing.....
Wed Sep  7 12:18:27 2022 INFO Start to run GSEA...Might take a while..................
Wed Sep  7 12:18:40 2022 INFO Start to generate gseapy reports, and produce figures...
Wed Sep  7 12:18:40 2022 INFO Congratulations. GSEApy runs successfully................

Wed Sep  7 12:18:40 2022 INFO Parsing data files for GSEA.............................


mowgli 4


KeyboardInterrupt: 

In [ ]:
enr_total.sort_values("pval").head(20)

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
regulation of transcription by RNA polymerase II (GO:0006357),0.903476,1.079339,0.000000,0.577960,2206,199,EZR;ZEB1;BACH2;BCL11B;RORA;CTNNB1;CAMK2D;NAMPT...,EZR;ZEB1;BACH2;BCL11B;RORA;CTNNB1;CAMK2D;NAMPT...
"regulation of transcription, DNA-templated (GO:0006355)",0.893007,1.066296,0.000000,0.589004,2244,190,ZEB1;BACH2;BCL11B;RORA;CAMK4;CTNNB1;CAMK2D;ATF...,ZEB1;BACH2;BCL11B;RORA;CAMK4;CTNNB1;CAMK2D;ATF...
positive regulation of transcription by RNA polymerase II (GO:0045944),0.902097,1.075641,0.000000,0.583348,908,111,BCL11B;RORA;CTNNB1;NAMPT;WWOX;PPP3CA;SQSTM1;NR...,BCL11B;RORA;CTNNB1;NAMPT;WWOX;PPP3CA;SQSTM1;NR...
"positive regulation of transcription, DNA-templated (GO:0045893)",0.894507,1.065982,0.004000,0.587943,1183,145,BCL11B;RORA;CAMK4;CTNNB1;NAMPT;WWOX;ATF7IP;PPP...,BCL11B;RORA;CAMK4;CTNNB1;NAMPT;WWOX;ATF7IP;PPP...
negative regulation of cell migration (GO:0030336),0.946804,1.121800,0.008000,0.915898,144,30,LDLRAD4;FOXO3;CD74;DUSP10;PTPRJ;CLIC4;TRIB1;NE...,LDLRAD4;FOXO3;CD74;DUSP10;PTPRJ
"negative regulation of transcription, DNA-templated (GO:0045892)",0.903699,1.072766,0.012000,0.586408,948,94,EZR;ZEB1;CTNNB1;WWOX;ATF7IP;JARID2;NR4A2;FOXO3...,EZR;ZEB1;CTNNB1;WWOX;ATF7IP;JARID2;NR4A2;FOXO3...
negative regulation of angiogenesis (GO:0016525),0.976821,1.190925,0.012048,0.772374,87,16,PDE3B;CTNNB1;KLF4;EPN2;TNF;NIBAN2;PTPRM;PPARG;...,PDE3B;CTNNB1
positive regulation of macromolecule biosynthetic process (GO:0010557),0.957229,1.147420,0.016000,1.000000,129,21,NAMPT;HSPH1;FCER2;KLF4;SLC25A37;TLR2;NIBAN1;SO...,NAMPT;HSPH1
sensory perception of sound (GO:0007605),0.979085,1.187040,0.016000,0.526255,91,15,CDC14A;CCDC50;ROR1;COL4A3;COL1A1;PJVK;CLIC5;SP...,CDC14A
